In [2]:
import pandas as pd
import re

# GCASH

In [7]:
# Load the original data
file_path = r"E:\GCASH.xlsx"
data = pd.read_excel(file_path)
data.head()

,Date and Time Description Reference No. Debit Credit Balance
0,2024-03-07 05:41 PM Withdraw from GSave Accoun...
1,2024-03-07 06:32 PM Payment to Uniku Taiwanese...
2,2024-03-07 08:21 PM Received GCash from 7eleve...
3,2024-03-08 02:26 PM Transfer from 09055149244 ...
4,2024-03-08 17:24:00


In [12]:
# Initialize an empty DataFrame to store the date_time and temp_description
details_data = pd.DataFrame(columns=["Date and Time", "Temporary Description"])

def extract_details(row):
    # Extract date and time
    date_time_pattern = r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}(?::\d{2})?(?: [AP]M)?'
    date_time_match = re.search(date_time_pattern, row)
    date_time = date_time_match.group(0) if date_time_match else ''

    # Extract description by removing the date and time part from the row
    description = row.replace(date_time, '').strip() if date_time else row

    return date_time, description

previous_description = ""
for i in range(len(data)):
    row = str(data.iloc[i, 0])

    # Skip rows containing 'Date and Time'
    if 'Date and Time' in row:
        continue

    date_time, temp_description = extract_details(row)

    if date_time:
        # If date_time is extracted, it's a new entry
        if previous_description:
            # Save the previous entry before starting a new one
            temp_df = pd.DataFrame([[None, previous_description]], columns=["Date and Time", "Temporary Description"])
            details_data = pd.concat([details_data, temp_df], ignore_index=True)
            previous_description = ""

        # Create a new entry for the current row
        temp_df = pd.DataFrame([[date_time, temp_description]], columns=["Date and Time", "Temporary Description"])
        details_data = pd.concat([details_data, temp_df], ignore_index=True)
    else:
        # If no date_time is found, append the row to the previous description
        previous_description += " " + row

# Add the last accumulated description if it exists
if previous_description:
    temp_df = pd.DataFrame([[None, previous_description]], columns=["Date and Time", "Temporary Description"])
    details_data = pd.concat([details_data, temp_df], ignore_index=True)

# Iterate through the details_data DataFrame to modify or drop rows based on conditions
for i in range(1, len(details_data)):
    if pd.isnull(details_data.at[i, 'Date and Time']):
        if 'Date and Time Description' in details_data.at[i, 'Temporary Description']:
            # Drop the row if 'Temporary Description' contains 'Date and Time Description'
            details_data.drop(i, inplace=True)
        else:
            # Append the current row's 'Temporary Description' to the previous row's 'Temporary Description'
            details_data.at[i - 1, 'Temporary Description'] += ' ' + details_data.at[i, 'Temporary Description']
            # Drop the current row after appending its description to the previous row
            details_data.drop(i, inplace=True)

# Reset the index after dropping rows
details_data.reset_index(drop=True, inplace=True)



details_data['Reference No.'] = None
details_data['Credit/Debit'] = None
details_data['Debit'] = None
details_data['Credit'] = None
details_data['Balance'] = None

# Iterate through the DataFrame to extract and assign the values to the new columns
for i in range(len(details_data)):
    # Split the 'Temporary Description' into blocks
    blocks = details_data.at[i, 'Temporary Description'].split()
    
    # Ensure there are enough blocks to extract the data
    if len(blocks) >= 3:
        details_data.at[i, 'Reference No.'] = blocks[-3]  # Third last block for 'Reference No.'
        details_data.at[i, 'Credit/Debit'] = blocks[-2]  # Second last block for 'Credit/Debit'
        details_data.at[i, 'Balance'] = blocks[-1]  # Last block for 'Balance'
        
        # Update 'Temporary Description' by removing the extracted parts
        details_data.at[i, 'Temporary Description'] = ' '.join(blocks[:-3])


# Add new columns for Credit and Debit


# Define keywords and phrases that indicate a credit transaction
credit_keywords = ['received', 'withdraw from', 'refund', 'transfer from system to']
transfer_from_ending = '09055149244'

# Iterate through the DataFrame to assign values to the Credit and Debit columns
for i in range(len(details_data)):
    description = details_data.at[i, 'Temporary Description'].lower()  # Convert to lowercase for case-insensitive comparison
    amount = details_data.at[i, 'Credit/Debit']
    description_blocks = description.split()
    
    # Check the conditions for credit
    if any(keyword in description for keyword in credit_keywords) or ('transfer from' in description and description_blocks[-1] == transfer_from_ending):
        details_data.at[i, 'Credit'] = amount
    else:
        details_data.at[i, 'Debit'] = amount

# Drop the 'Credit/Debit' column as it's now separated
details_data.drop('Credit/Debit', axis=1, inplace=True)

# Separate 'Date and Time' into two new columns: 'Date' and 'Time'
details_data[['Date', 'Time']] = details_data['Date and Time'].str.split(' ', n=1, expand=True)

from datetime import datetime

def convert_time(time_str):
    # Check if the time is already in 12-hour format
    if 'AM' in time_str or 'PM' in time_str:
        return time_str
    else:
        # Convert from 24-hour to 12-hour format
        return datetime.strptime(time_str, '%H:%M:%S').strftime('%I:%M %p')

# Apply the conversion function to the 'Time' column
details_data['Time'] = details_data['Time'].apply(convert_time)


# Reorder the DataFrame to place the 'Date' and 'Time' columns at the front
column_order = ['Date', 'Time', 'Temporary Description', 'Reference No.', 'Credit', 'Debit', 'Balance']
details_data = details_data[column_order]



# Convert the 'Credit' and 'Debit' columns to numeric, ensuring non-numeric values are handled as NaN
details_data['Credit'] = pd.to_numeric(details_data['Credit'], errors='coerce').fillna(0)
details_data['Debit'] = pd.to_numeric(details_data['Debit'], errors='coerce').fillna(0)

# Calculate the total credit and total debit
total_credit = details_data['Credit'].sum()
total_debit = details_data['Debit'].sum()

# Determine the ending balance
# If the 'Balance' column represents the running balance, take the last entry
# Otherwise, calculate it as total credit minus total debit
if pd.to_numeric(details_data['Balance'], errors='coerce').notnull().all():
    ending_balance = details_data['Balance'].astype(float).iloc[-1]
else:
    ending_balance = total_credit - total_debit

# Print the results
print(f"Total Credit: {total_credit}")
print(f"Total Debit: {total_debit}")
print(f"Ending Balance: {ending_balance}")

#Show the updated DataFrame
details_data


Total Credit: 56118.51
Total Debit: 55551.33
Ending Balance: 929.63


,Date,Time,Temporary Description,Reference No.,Credit,Debit,Balance
0,2024-03-07,05:41 PM,Withdraw from GSave Account with Reference No....,6015793528450,1000.00,0.00,1362.45
1,2024-03-07,06:32 PM,Payment to Uniku Taiwanese Restaurant,0000614550018,0.00,420.00,942.45
2,2024-03-07,08:21 PM,Received GCash from 7eleven Kiosk,6015797770799,2000.79,0.00,2943.24
3,2024-03-08,02:26 PM,Transfer from 09055149244 to 09264080098,6015813258374,0.00,80.00,2863.24
4,2024-03-08,05:24 PM,"Payment to Google, Merchant Transaction Number...",0000616013653,0.00,49.00,2814.24
5,2024-03-08,07:10 PM,"Payment to Google, Merchant Transaction Number...",0000616246561,0.00,149.00,2665.24
6,2024-03-09,11:31 AM,Payment to Subway 1961,0000617186903,0.00,715.00,1950.24
7,2024-03-09,01:04 PM,Payment to Bancnet P2M Send,0000617314087,0.00,800.00,1150.24
8,2024-03-09,01:09 PM,Transfer from 09989195697 to 09055149244,9015836236735,2350.00,0.00,3500.24
9,2024-03-09,01:23 PM,Payment to Bancnet P2M Send,0000617344331,0.00,180.00,3320.24


In [13]:
details_data.dtypes

Date                      object
Time                      object
Temporary Description     object
Reference No.             object
Credit                   float64
Debit                    float64
Balance                   object
dtype: object

In [15]:
# Save the DataFrame to an Excel file without the index
output_file_path = 'gcash_data_0307_0404.xlsx'
details_data.to_excel(output_file_path, index=False)

# BDO

In [24]:
# Load the original data
file_path = r"E:\BDO.xlsx"
data = pd.read_excel(file_path)

In [25]:
data.head()

,Sale Date Post Date Transaction Details Amount
0,03/21/24 03/21/24 MONTHLY MEMBERSHIP FEE 150.00
1,02/22/24 02/22/24 SM STORE-MEGAMALL MANDALUYON...
2,02/27/24 02/28/24 SKY CABLE 3D IPG QC MPGS QUE...
3,03/01/24 03/03/24 ITALIANNIS SHANGRILA M MANDA...
4,03/01/24 03/03/24 MARY GRACE SHNGRILA MANDALUY...


In [26]:
# Initialize an empty DataFrame to store the date_time and temp_description
details_data = pd.DataFrame(columns=["Date and Time", "Temporary Description"])

def extract_details(row):
    # Extract date and time
    date_time_pattern = r'\d{2}/\d{2}/\d{2}'
    date_time_match = re.search(date_time_pattern, row)
    date_time = date_time_match.group(0) if date_time_match else ''

    # Extract description by removing the date and time part from the row
    description = row.replace(date_time, '').strip() if date_time else row

    return date_time, description

previous_description = ""
for i in range(len(data)):
    row = data.iloc[i, 0]

    # Skip rows containing 'Date and Time'
    if 'Date and Time' in row:
        continue

    date_time, temp_description = extract_details(row)

    if date_time:
        # If date_time is extracted, it's a new entry
        if previous_description:
            # Save the previous entry before starting a new one
            temp_df = pd.DataFrame([[None, previous_description]], columns=["Date and Time", "Temporary Description"])
            details_data = pd.concat([details_data, temp_df], ignore_index=True)
            previous_description = ""

        # Create a new entry for the current row
        temp_df = pd.DataFrame([[date_time, temp_description]], columns=["Date and Time", "Temporary Description"])
        details_data = pd.concat([details_data, temp_df], ignore_index=True)
    else:
        # If no date_time is found, append the row to the previous description
        previous_description += " " + row

# Add the last accumulated description if it exists
if previous_description:
    temp_df = pd.DataFrame([[None, previous_description]], columns=["Date and Time", "Temporary Description"])
    details_data = pd.concat([details_data, temp_df], ignore_index=True)

# Iterate through the details_data DataFrame to modify or drop rows based on conditions
for i in range(1, len(details_data)):
    if pd.isnull(details_data.at[i, 'Date and Time']):
        if 'Date and Time Description' in details_data.at[i, 'Temporary Description']:
            # Drop the row if 'Temporary Description' contains 'Date and Time Description'
            details_data.drop(i, inplace=True)
        else:
            # Append the current row's 'Temporary Description' to the previous row's 'Temporary Description'
            details_data.at[i - 1, 'Temporary Description'] += ' ' + details_data.at[i, 'Temporary Description']
            # Drop the current row after appending its description to the previous row
            details_data.drop(i, inplace=True)

# Reset the index after dropping rows
details_data.reset_index(drop=True, inplace=True)





In [27]:
# Update the function to capture negative amounts as well
def extract_details_with_negative_amount(row):
    description = row['Temporary Description']
    sale_date = row['Date and Time']
    
    # Initialize default values
    post_date, desc, amount = None, description, None
    
    # Check if 'reference' is in the description
    if 'reference' in description.lower():
        # Pattern when 'reference' is present: extract the last number as amount (considering commas and negatives) and remove it from the description
        amount_parts = re.findall(r'-?[\d,]+\.\d+', description)
        if amount_parts:
            amount = amount_parts[-1].replace(',', '')
            desc = description.replace(amount_parts[-1], '').strip()
            post_date_match = re.search(r'\d{2}/\d{2}/\d{2}', desc)
            post_date = post_date_match.group() if post_date_match else sale_date
            desc = desc.replace(post_date, '').strip() if post_date_match else desc
    else:
        # Default pattern: extract the last number as amount (considering commas and negatives) and remove it from the description
        amount_parts = re.findall(r'-?[\d,]+\.\d+', description)
        if amount_parts:
            amount = amount_parts[-1].replace(',', '')
            desc = description.replace(amount_parts[-1], '').strip()
            post_date_match = re.search(r'\d{2}/\d{2}/\d{2}', desc)
            post_date = post_date_match.group() if post_date_match else sale_date
            desc = desc.replace(post_date, '').strip() if post_date_match else desc
    
    return pd.Series([sale_date, post_date, desc, amount], index=['Sale Date', 'Post Date', 'Description', 'Amount'])

# Apply the updated function to the original data
excel_data = details_data.copy()
extracted_data_with_negative = excel_data.apply(extract_details_with_negative_amount, axis=1)

# Show the modified dataframe
extracted_data_with_negative.head()


,Sale Date,Post Date,Description,Amount
0,03/21/24,03/21/24,MONTHLY MEMBERSHIP FEE,150.00
1,02/22/24,02/22/24,SM STORE-MEGAMALL MANDALUYONG PH,4798.40
2,02/27/24,02/28/24,SKY CABLE 3D IPG QC MPGS QUEZON CITY PH,1299.00
3,03/01/24,03/03/24,ITALIANNIS SHANGRILA M MANDALUYONG PH,2203.28
4,03/01/24,03/03/24,MARY GRACE SHNGRILA MANDALUYONG PH,212.00


In [28]:
extracted_data_with_negative

,Sale Date,Post Date,Description,Amount
0,03/21/24,03/21/24,MONTHLY MEMBERSHIP FEE,150.00
1,02/22/24,02/22/24,SM STORE-MEGAMALL MANDALUYONG PH,4798.40
2,02/27/24,02/28/24,SKY CABLE 3D IPG QC MPGS QUEZON CITY PH,1299.00
3,03/01/24,03/03/24,ITALIANNIS SHANGRILA M MANDALUYONG PH,2203.28
4,03/01/24,03/03/24,MARY GRACE SHNGRILA MANDALUYONG PH,212.00
5,03/11/24,03/11/24,THE MARKETPLACE SHANGR MANDALUYONG PH,1416.25
6,03/14/24,03/14/24,PAYMENT RECEIVED - THANK YOU,-9500.00
7,03/14/24,03/17/24,S&R PIZZA AYALA THE 30TH PASIG PH Reference:...,1875.30
8,03/15/24,03/17/24,PYM*WHEYL NUTRITION CO. Taguig City PH Refer...,2848.00
9,03/17/24,03/17/24,WATSONS SHANGRILA PLAZ MANDALUYONG PH,245.00


In [29]:
# Save the DataFrame to an Excel file without the index
output_file_path = 'bdo_mar.xlsx'
extracted_data_with_negative.to_excel(output_file_path, index=False)

# UB

In [9]:
# Load the original data
file_path = r"E:\UB_MAR1_SOA.xlsx"
data = pd.read_excel(file_path, sheet_name='Sheet2', engine='openpyxl')

In [10]:
data.head()

,Transaction_Date Posting_Date Transaction_Details Amount
0,04/02/2024 04/02/2024 U‐PROTECT Premium CHARGE...
1,"03/02/2024 03/04/2024 GADC MATALINO 193, QUEZO..."
2,"03/02/2024 03/04/2024 LAZADA PH, MAKATI 3,998.02"
3,"03/03/2024 03/05/2024 LAZADA PH, MAKATI 1,337.38"
4,"03/04/2024 03/05/2024 LAZADA PH, MAKATI 726.68"


In [11]:
# Re-initialize the details_data DataFrame to ensure it's empty before applying the transformation again
details_data = pd.DataFrame(columns=['Transaction Date', 'Posting Date', 'Transaction Detail', 'Amount'])

# Initialize a list to store each row's data as a dictionary
rows_list = []

# Iterate over each row in Sheet 1 to extract and transform data
for index, row in data.iterrows():
    # Split the row by space
    split_data = row[0].split()
    
    # Extracting individual elements based on the provided logic
    transaction_date = split_data[0]
    posting_date = split_data[1]
    amount = split_data[-1]
    transaction_detail = ' '.join(split_data[2:-1])  # Concatenating the remaining data as transaction detail
    
    # Appending the extracted data as a dictionary to the rows list
    rows_list.append({
        'Transaction Date': transaction_date,
        'Posting Date': posting_date,
        'Transaction Detail': transaction_detail,
        'Amount': amount
    })

# Concatenating the list of dictionaries as rows to the DataFrame
details_data = pd.concat([details_data, pd.DataFrame(rows_list)], ignore_index=True)

details_data


,Transaction Date,Posting Date,Transaction Detail,Amount
0,04/02/2024,04/02/2024,U‐PROTECT Premium CHARGE,501.18
1,03/02/2024,03/04/2024,"GADC MATALINO 193, QUEZON CITY","1,081.00"
2,03/02/2024,03/04/2024,"LAZADA PH, MAKATI","3,998.02"
3,03/03/2024,03/05/2024,"LAZADA PH, MAKATI","1,337.38"
4,03/04/2024,03/05/2024,"LAZADA PH, MAKATI",726.68
5,03/04/2024,03/05/2024,"GRAB, MAKATI","1,470.00"
6,03/05/2024,03/06/2024,"GRAB, MAKATI","1,301.00"
7,03/07/2024,03/07/2024,(06/36) OGAWA_SHANGRI LA,"2,777.78"
8,03/07/2024,03/08/2024,"STARBUCKS 486 ZETA TO, QUEZON CITY",190.00
9,03/08/2024,03/09/2024,"SHAKEYS, PARANAQUE","1,455.43"


In [14]:
df = details_data.copy()

In [20]:
import re

# Function to replace non-standard hyphens/dashes with a standard hyphen
def replace_non_standard_hyphens(s):
    return re.sub(r'[^\d.,]+', '-', s)

# Apply the function to your column and then convert to float
details_data['Amount'] = details_data['Amount'].apply(replace_non_standard_hyphens).str.replace(',', '').astype(float)


In [21]:
details_data['Amount'].sum() 

45811.82000000001

In [23]:
# Specify the filename for the output Excel file
output_file_path = r"E:\UB_APR1_SOA_transformed.xlsx"

# Write the DataFrame to an Excel file without the index
details_data.to_excel(output_file_path, index=False)
